# First Getting the face recognition Code

### A. Loading the Model of face recognition

In [3]:
import cv2
import numpy as np
import pickle
import keras

mp = keras.models.load_model("Models/FaceRecognitionModel.keras")
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("Models/ModelToRecogniseUnrecogniseFace.yml")

# Load the face classifier
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load the face map for label mapping
with open("ResultsMap.pkl", 'rb') as fileReadStream:
    ResultMap = pickle.load(fileReadStream)

print(ResultMap)
# Initialize the webcam
webcam = cv2.VideoCapture(0)

# Function to recognize faces
def recognize_faces(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    name = ''
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = frame[y:y+h, x:x+w]
        face_roi1 = gray[y:y+h, x:x+w]
        
        label_id, confidence = recognizer.predict(face_roi1)

        # Resize the face ROI to match the input size of the model
        resized_face = cv2.resize(face_roi, (64, 64))

        # Preprocess the face (normalize pixel values)
        normalized_face = resized_face / 255.0

        # Reshape the face for the model input (add batch dimension)
        reshaped_face = np.expand_dims(normalized_face, axis=0)

        # Perform prediction using the model
        prediction = mp.predict(reshaped_face)
        predicted_label = ResultMap[np.argmax(prediction)]
        print('confidence',prediction[0][np.argmax(prediction)])
        print(prediction)
        # print(predicted_label)
        print(np.max(prediction))
        name = predicted_label
        pre = np.max(prediction)
        print(confidence)
        if confidence > 110.07375104700013: 
            cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2) 
            name = 'Unknown'
        else:
            cv2.putText(frame, predicted_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)


        # Draw the bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        

    return frame , name

# Main loop to capture frames from the webcam
while True:
    # Read a frame from the webcam
    ret, frame = webcam.read()
    if not ret:
        break

    # Perform face recognition on the frame
    frame , name = recognize_faces(frame)
    print(name)
    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Check for 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
webcam.release()
cv2.destroyAllWindows()




{0: 'Pushkar', 1: 'Sairaj', 2: 'Tushar'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
confidence 0.3554518
[[0.3126563  0.3554518  0.33189198]]
0.3554518
131.89836327571754
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
confidence 0.34757784
[[0.3200859  0.34757784 0.33233634]]
0.34757784
55.370726058142495
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
confidence 0.35296786
[[0.31378102 0.35296786 0.33325115]]
0.35296786
172.56663698480182
Unknown
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
confidence 0.35641283
[[0.3139301  0.35641283 0.32965705]]
0.35641283
134.35094835941243
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
confidence 0.34911558
[[0.32000488 0.34911558 0.33087954]]
0.34911558
53.75867339123394
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
confidence 0.35643542
[[0.31320032 0.35643542 0.33036426]]
0.35643542
172.26359577869886
Unknown
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
confidence 0.35375863
[[0.31442976 0.35375863 0.33181164]]
0.35375863
173.53302303821516
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
confidence 0.35756

## B. Loading the model of the action recongition

In [4]:
import keras
from keras import layers, Sequential
from keras.models import load_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import cv2

import mediapipe as mp
import glob
import pandas as pd
import numpy as np
import math
import os 


# ap = argparse.ArgumentParser()
# ap.add_argument("-m", "--model", type=str, required=True,
#                 help="path to saved .h5 model, eg: dir/model.h5")
# ap.add_argument("-c", "--conf", type=float, required=True,
#                 help="min prediction conf to detect pose class (0<conf<1)")
# ap.add_argument("-i", "--source", type=str, required=True,
#                 help="path to sample image")
# ap.add_argument("--save", action='store_true',
#                 help="Save video")

# args = vars(ap.parse_args())
source = "Video_5.mp4"
path_saved_model = "Models/Action Recognition/model.keras"
threshold = 0.2
save = "Output"
print('Hello')
##############
torso_size_multiplier = 2.5
n_landmarks = 33
n_dimensions = 3
landmark_names = [
    'nose',
    'left_eye_inner', 'left_eye', 'left_eye_outer',
    'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear',
    'mouth_left', 'mouth_right',
    'left_shoulder', 'right_shoulder',
    'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist',
    'left_pinky_1', 'right_pinky_1',
    'left_index_1', 'right_index_1',
    'left_thumb_2', 'right_thumb_2',
    'left_hip', 'right_hip',
    'left_knee', 'right_knee',
    'left_ankle', 'right_ankle',
    'left_heel', 'right_heel',
    'left_foot_index', 'right_foot_index',
]
class_names = [
    'Running','Standing','Walking','Waving'
]
##############

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

col_names = []
for i in range(n_landmarks):
    name = mp_pose.PoseLandmark(i).name
    name_x = name + '_X'
    name_y = name + '_Y'
    name_z = name + '_Z'
    name_v = name + '_V'
    col_names.append(name_x)
    col_names.append(name_y)
    col_names.append(name_z)
    col_names.append(name_v)

# Load saved model
model = load_model(path_saved_model, compile=True)

if source.endswith(('.jpg', '.jpeg', '.png')):
    path_to_img = source
    print('Entered1')
    # Load sample Image
    
    img = cv2.imread(path_to_img)
    print(img)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = pose.process(img_rgb)
    if result.pose_landmarks:
        lm_list = []
        for landmarks in result.pose_landmarks.landmark:
            # Preprocessing
            max_distance = 0
            lm_list.append(landmarks)
        print('landmarks')
        print(landmarks)
        center_x = (lm_list[landmark_names.index('right_hip')].x +
                    lm_list[landmark_names.index('left_hip')].x)*0.5
        center_y = (lm_list[landmark_names.index('right_hip')].y +
                    lm_list[landmark_names.index('left_hip')].y)*0.5

        shoulders_x = (lm_list[landmark_names.index('right_shoulder')].x +
                       lm_list[landmark_names.index('left_shoulder')].x)*0.5
        shoulders_y = (lm_list[landmark_names.index('right_shoulder')].y +
                       lm_list[landmark_names.index('left_shoulder')].y)*0.5

        for lm in lm_list:
            distance = math.sqrt((lm.x - center_x)**2 + (lm.y - center_y)**2)
            if(distance > max_distance):
                max_distance = distance
        torso_size = math.sqrt((shoulders_x - center_x) **
                               2 + (shoulders_y - center_y)**2)
        max_distance = max(torso_size*torso_size_multiplier, max_distance)

        pre_lm = list(np.array([[(landmark.x-center_x)/max_distance, (landmark.y-center_y)/max_distance,
                                 landmark.z/max_distance, landmark.visibility] for landmark in lm_list]).flatten())
        data = pd.DataFrame([pre_lm], columns=col_names)
        print(data)
        predict = model.predict(data)[0]
        print('predicted')
        if max(predict) > threshold:
            pose_class = class_names[predict.argmax()]
            print('predictions: ', predict)
            print('predicted Pose Class: ', pose_class)
        else:
            pose_class = 'Unknown Pose'
            print('[INFO] Predictions is below given Confidence!!')

    # Show Result
    img = cv2.putText(
        img, f'{class_names[predict.argmax()]}',
        (40, 50), cv2.FONT_HERSHEY_PLAIN,
        2, (255, 0, 255), 2
    )

    if save:
        os.makedirs('ImageOutput', exist_ok=True)
        img_full_name = os.path.split(path_to_img)[1]
        img_name = os.path.splitext(img_full_name)[0]
        path_to_save_img = f'ImageOutput/{img_name}.jpg'
        cv2.imwrite(f'{path_to_save_img}', img)
        print(f'[INFO] Output Image Saved in {path_to_save_img}')

    cv2.imshow('Output Image', img)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
    print('[INFO] Inference on Test Image is Ended...')

else:
    # Web-cam
    if source.isnumeric():
        source = int(source)

    cap = cv2.VideoCapture(source)
    source_width = int(cap.get(3))
    source_height = int(cap.get(4))

    # Write Video
    if save:
        out_video = cv2.VideoWriter('output.avi', 
                            cv2.VideoWriter_fourcc(*'MJPG'),
                            10, (source_width, source_height))

    while True:
        success, img = cap.read()
        if not success:
            print('[ERROR] Failed to Read Video feed')
            break
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = pose.process(img_rgb)

        if result.pose_landmarks:
            lm_list = []
            for landmarks in result.pose_landmarks.landmark:
                # Preprocessing
                max_distance = 0
                lm_list.append(landmarks)
            center_x = (lm_list[landmark_names.index('right_hip')].x +
                        lm_list[landmark_names.index('left_hip')].x)*0.5
            center_y = (lm_list[landmark_names.index('right_hip')].y +
                        lm_list[landmark_names.index('left_hip')].y)*0.5

            shoulders_x = (lm_list[landmark_names.index('right_shoulder')].x +
                           lm_list[landmark_names.index('left_shoulder')].x)*0.5

            
            shoulders_y = (lm_list[landmark_names.index('right_shoulder')].y +
                           lm_list[landmark_names.index('left_shoulder')].y)*0.5

            for lm in lm_list:
                distance = math.sqrt((lm.x - center_x) **
                                     2 + (lm.y - center_y)**2)
                if(distance > max_distance):
                    max_distance = distance
            torso_size = math.sqrt((shoulders_x - center_x) **
                                   2 + (shoulders_y - center_y)**2)
            max_distance = max(torso_size*torso_size_multiplier, max_distance)

            pre_lm = list(np.array([[(landmark.x-center_x)/max_distance, (landmark.y-center_y)/max_distance,
                                     landmark.z/max_distance, landmark.visibility] for landmark in lm_list]).flatten())
            data = pd.DataFrame([pre_lm], columns=col_names)
            # print(data[2])
            predict = model.predict(data)[0]
            if max(predict) > threshold:
                pose_class = class_names[predict.argmax()]
                print('predictions: ', predict)
                print('predicted Pose Class: ', pose_class)
            else:
                pose_class = 'Unknown Pose'
                print('[INFO] Predictions is below given Confidence!!')

            # Show Result
            img = cv2.putText(
                img, f'{pose_class}',
                (40, 50), cv2.FONT_HERSHEY_PLAIN,
                2, (255, 0, 255), 2
            )
        # Write Video
        if save:
            out_video.write(img)

        cv2.imshow('Output Image', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    if save:
        out_video.release()
        print("[INFO] Out video Saved as 'output.avi'")
    cv2.destroyAllWindows()
    print('[INFO] Inference on Videostream is Ended...')

Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
predictions:  [4.1327358e-04 9.9323118e-01 6.1787656e-03 1.7673564e-04]
predicted Pose Class:  Standing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
predictions:  [5.9373397e-04 9.9327725e-01 6.0086939e-03 1.2034838e-04]
predicted Pose Class:  Standing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
predictions:  [7.9373072e-04 9.9275851e-01 6.3235676e-03 1.2420100e-04]
predicted Pose Class:  Standing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
predictions:  [6.0241087e-04 9.9399370e-01 5.2793478e-03 1.2459271e-04]
predicted Pose Class:  Standing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
predictions:  [7.3159731e-04 9.9471986e-01 4.4303038e-03 1.1816680e-04]
predicted Pose Class:  Standing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
predictions:  [1.3630603e-03 9.9259514e-01 5.9079872e-03 1.3388120e-04]
predicted Pose Class:  Standing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
predictions:  [1.8013919e-03 9.9166846e-01 6.4050741e-03 1.2502850e-04]
predicted Pose Class:  Standing


## C. Intrusion Detection code

In [8]:
import pickle
from collections import Counter
import keras
import numpy as np
import cv2
import time
import datetime
from collections import deque
from twilio.rest import Client 
import random
import heapq
import os
import matplotlib.pyplot as plt
import random
import requests
import keras
from keras import layers, Sequential
from keras.models import load_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import mediapipe as mp
import glob
import pandas as pd
import math

In [9]:


def is_person_present(frame, thresh=1100):
    kernel = None
    global foog
    
    # Apply background subtraction
    fgmask = foog.apply(frame)

    # Get rid of the shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask
    fgmask = cv2.dilate(fgmask, kernel, iterations=4)

    # Detect contours in the frame
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
     
    # Check if there was a contour and the area is somewhat higher than some threshold so we know it's a person and not noise
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > thresh:
            
        # Get the max contour
        cnt = max(contours, key=cv2.contourArea)

        # Draw a bounding box around the person and label it as person detected
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, 'Person Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
        
        return True, frame
        
        
    # Otherwise report there was no one present
    else:
        return False, frame


def send_message(body, info_dict):

    # Your Account SID from twilio.com/console
    account_sid = info_dict['account_sid']

    # Your Auth Token from twilio.com/console
    auth_token  = info_dict['auth_token']


    client = Client(account_sid, auth_token)

    message = client.messages.create(to=info_dict['your_num'], from_=info_dict['trial_num'], body=body)





def calculate_percentage_change(frame1, frame2):
    # Convert frames to grayscale
    gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate absolute difference between the two frames
    abs_diff = cv2.absdiff(gray_frame1, gray_frame2)

    # Calculate percentage change
    percentage_change = (np.count_nonzero(abs_diff) / abs_diff.size) * 100

    return percentage_change


def save_frame(frame_heap): 
    frame_index = 0
    while frame_heap:
        # Pop the frame with the maximum percentage change (negated)
        neg_change_percentage, frame , date = heapq.heappop(frame_heap)
        change_percentage = -neg_change_percentage
        file_path = 'DetectedPerson'
        # Save the frame to disk
        cv2.imwrite(f'{file_path}/{date}.jpg', frame)
        frame_index += 1
    print("saved frame succesfully")


    


print('no error in functions')

no error in functions


## Final Code

In [10]:
# Set Window normal so we can resize it
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# This is a test video
cap = cv2.VideoCapture('sample_video1.mp4')

# Read the video stream from the camera
# cap = cv2.VideoCapture('http://192.168.43.1:8080/video')

# Get width and height of the frame
width = int(cap.get(3))
height = int(cap.get(4))

# Read and store the credentials information in a dict
with open('credential.txt', 'r') as myfile:
  data = myfile.read()

info_dict = eval(data)

# Initialize the background Subtractor
foog = cv2.createBackgroundSubtractorMOG2(detectShadows=True, varThreshold=100, history=2000)

# Status is True when person is present and False when the person is not present.
status = False

# After the person disappears from view, wait at least 7 seconds before making the status False
patience = 0.099

# We don't consider an initial detection unless it's detected 15 times, this gets rid of false positives
detection_thresh = 5

# Initial time for calculating if patience time is up
initial_time = None



# We are creating a deque object of length detection_thresh and will store individual detection statuses here
de = deque([False] * 10, maxlen=10)

# Initialize an empty heap to store frames along with their negated percentage change
frame_heap = []


# Initialize these variables for calculating FPS
fps = 0 
frame_counter = 0
start_time = time.time()
counter = 0

ret,prevframe = cap.read()
initial_frame = prevframe
recognisedName = ""
sent  = False

while True:
    
    ret, frame = cap.read()
    if not ret:
        break 
    
            
    # This function will return a boolean variable telling if someone was present or not, it will also draw boxes if it 
    # finds someone
    detected, annotated_image = is_person_present(frame)  
    # print(recognisedName)
    if detected and len(recognisedName) == 0: 
        # result = recogniseFromFace(frame)  
        result = ''
        if result is not None: 
            result = ''
    # Register the current detection status on our deque object
    de.appendleft(detected)
    
    # If we have consecutively detected a person 15 times then we are sure that someone is present    
    # We also make this is the first time that this person has been detected so we only initialize the videowriter once
    if sum(de) == detection_thresh and not status:                       
            status = True
            entry_time = datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")
            # out = cv2.VideoWriter('outputs/{}.mp4'.format(entry_time), cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))

    # If status is True but the person is not in the current frame
    if status and not detected:
        
        # Restart the patience timer only if the person has not been detected for a few frames so we are sure it wasn't a 
        # False positive
        print('i reached deepest of the darkest point')
        if sum(de) > (detection_thresh/2): 
            if initial_time is None:
                print('initial time as been setted to',initial_time)
                initial_time = time.time()
                
            
            elif initial_time is not None:        
                print('now the initial time is not null beacuse current time is ',initial_time)
                print(time.time() - initial_time)
                # If the patience has run out and the person is still not detected then set the status to False
                # Also save the video by releasing the video writer and send a text message.
                if  time.time() - initial_time >= patience:
                    status = False
                    print("inside the loop")
                    exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
                    # out.release()
                    # print(out)
                    initial_time = None
                    de = deque([False] * len(de))
                    body = "Alert: \nA Person Entered the Room at {} \nLeft the room at {}".format(entry_time, exit_time) 
                    if len(recognisedName) > 0: 
                        body = "Alert: {} Has entered in your Room at {} \n Left the room at {}".format(recognisedName,entry_time,exit_time)
                    print('message has been sent')
                    send_message(body, info_dict)
                    frame_index = 0
                    cv2.destroyWindow("frame")
                    while frame_heap:
                        # Pop the frame with the maximum percentage change (negated)
                        neg_change_percentage, frame , date = heapq.heappop(frame_heap)
                        change_percentage = -neg_change_percentage
                        file_path = 'DetectedPerson'
                        
                        # Save the frame to disk
                        cv2.imwrite(f'{file_path}/{date}.jpg', frame)
                        frame_index += 1
                    print("saved frame succesfully")
                    continue
                    
                
    
    # If a significant amount of detections (more than half of detection_thresh) has occurred then we reset the Initial Time.
    elif status and sum(de) > (detection_thresh/2):
        change_percentage = calculate_percentage_change(prevframe, frame)
        change_initial_change = calculate_percentage_change(initial_frame,frame)
        # print(change_percentage)
        if change_percentage > 95 and sent == False: 
            sent = True
            body = "Alert: \n Some one has changed the camera configuration at:{}".format(datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")) 
            send_message(body,info_dict)
    
        if change_initial_change > 60: 
            heapq.heappush(frame_heap, (-change_percentage, frame,datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")))
        prevframe = frame
        initial_time = None
    
    # Get the current time in the required format
    current_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")

    # Display the FPS
    cv2.putText(annotated_image, 'FPS: {:.2f}'.format(fps), (510, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155), 2)
    
    # Display Time
    cv2.putText(annotated_image, current_time, (310, 20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1)    
    
    # Display the Room Status
    cv2.putText(annotated_image, 'Room Occupied: {}'.format(str(status)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                (200, 10, 150), 2)

    # Show the patience Value
    if initial_time is None:
        text = 'Patience: {}'.format(patience)
    else: 
        text = 'Patience: {:.2f}'.format(max(0, patience - (time.time() - initial_time)))
        
    cv2.putText(annotated_image, text, (10, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155), 2)   

 
    # Show the Frame
    cv2.imshow('frame', frame)
    
    # Calculate the Average FPS
    frame_counter += 1
    fps = (frame_counter / (time.time() - start_time))
    
    
    # Exit if q is pressed.
    if cv2.waitKey(30) == ord('q'):
        break


# Release Capture and destroy windows
cap.release()
cv2.destroyAllWindows()

i reached deepest of the darkest point
initial time as been setted to None
i reached deepest of the darkest point
now the initial time is not null beacuse current time is  1713391122.9648983
0.1489403247833252
inside the loop
message has been sent
saved frame succesfully


## D. Final Code Of ML

In [5]:
import pickle
from collections import Counter
import keras
import numpy as np
import cv2
import time
import datetime
from collections import deque
from twilio.rest import Client 
import random
import heapq
import os
import matplotlib.pyplot as plt
import random
import requests
import keras
from keras import layers, Sequential
from keras.models import load_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import mediapipe as mp
import glob
import pandas as pd
import math




print("Libraries succesfully imported") 

mp1 = keras.models.load_model("Models/FaceRecognitionModel.keras")


path_saved_model = "Models/Action Recognition/model.keras"
threshold = 0.2
save = "Output"
print('Hello')
##############
torso_size_multiplier = 2.5
n_landmarks = 33
n_dimensions = 3
landmark_names = [
    'nose',
    'left_eye_inner', 'left_eye', 'left_eye_outer',
    'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear',
    'mouth_left', 'mouth_right',
    'left_shoulder', 'right_shoulder',
    'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist',
    'left_pinky_1', 'right_pinky_1',
    'left_index_1', 'right_index_1',
    'left_thumb_2', 'right_thumb_2',
    'left_hip', 'right_hip',
    'left_knee', 'right_knee',
    'left_ankle', 'right_ankle',
    'left_heel', 'right_heel',
    'left_foot_index', 'right_foot_index',
]
class_names = [
    'Running','Standing','Walking','Waving'
]


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

col_names = []
for i in range(n_landmarks):
    name = mp_pose.PoseLandmark(i).name
    name_x = name + '_X'
    name_y = name + '_Y'
    name_z = name + '_Z'
    name_v = name + '_V'
    col_names.append(name_x)
    col_names.append(name_y)
    col_names.append(name_z)
    col_names.append(name_v)

# Load saved model
model = load_model(path_saved_model, compile=True)


# Load the face classifier
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load the face map for label mapping
with open("ResultsMap.pkl", 'rb') as fileReadStream:
    ResultMap = pickle.load(fileReadStream)

print(ResultMap)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("Models/ModelToRecogniseUnrecogniseFace.yml")


# Function to recognize faces
def recognize_faces(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    predicted_label = ''
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = frame[y:y+h, x:x+w]
        face_roi1 = gray[y:y+h, x:x+w]

        # Resize the face ROI to match the input size of the model
        label_id, confidence = recognizer.predict(face_roi1)

        # Resize the face ROI to match the input size of the model
        resized_face = cv2.resize(face_roi, (64, 64))

        # Preprocess the face (normalize pixel values)
        normalized_face = resized_face / 255.0

        # Reshape the face for the model input (add batch dimension)
        reshaped_face = np.expand_dims(normalized_face, axis=0)

        # Perform prediction using the model
        prediction = mp1.predict(reshaped_face)
        predicted_label = ResultMap[np.argmax(prediction)]
        print('confidence',prediction[0][np.argmax(prediction)])
        print(prediction)
        # print(predicted_label)
        print(np.max(prediction))
        name = predicted_label
        pre = np.max(prediction)
        print(confidence)
        if confidence > 70: 
            cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2) 
            name = 'Unknown'
        else:
            cv2.putText(frame, predicted_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        print(name)
        # Draw the bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return predicted_label , frame


def recognisize_pose(frame): 
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(img_rgb)
    activity = ''
    if result.pose_landmarks:
        lm_list = []
        for landmarks in result.pose_landmarks.landmark:
            # Preprocessing
            max_distance = 0
            lm_list.append(landmarks)
        # print('landmarks')
        # print(landmarks)
        center_x = (lm_list[landmark_names.index('right_hip')].x +
                    lm_list[landmark_names.index('left_hip')].x)*0.5
        center_y = (lm_list[landmark_names.index('right_hip')].y +
                    lm_list[landmark_names.index('left_hip')].y)*0.5

        shoulders_x = (lm_list[landmark_names.index('right_shoulder')].x +
                       lm_list[landmark_names.index('left_shoulder')].x)*0.5
        shoulders_y = (lm_list[landmark_names.index('right_shoulder')].y +
                       lm_list[landmark_names.index('left_shoulder')].y)*0.5

        for lm in lm_list:
            distance = math.sqrt((lm.x - center_x)**2 + (lm.y - center_y)**2)
            if(distance > max_distance):
                max_distance = distance
        torso_size = math.sqrt((shoulders_x - center_x) **
                               2 + (shoulders_y - center_y)**2)
        max_distance = max(torso_size*torso_size_multiplier, max_distance)

        pre_lm = list(np.array([[(landmark.x-center_x)/max_distance, (landmark.y-center_y)/max_distance,
                                 landmark.z/max_distance, landmark.visibility] for landmark in lm_list]).flatten())
        data = pd.DataFrame([pre_lm], columns=col_names)
        # print(data)
        predict = model.predict(data)[0]
        print('predicted')
        activity = class_names[predict.argmax()]
        if max(predict) > threshold:
            pose_class = class_names[predict.argmax()]
            print('predictions: ', predict)
            print('predicted Pose Class: ', pose_class)
        else:
            pose_class = 'Unknown Pose'
            print('[INFO] Predictions is below given Confidence!!')

        frame = cv2.putText(
                frame, f'{class_names[predict.argmax()]}',
                (40, 50), cv2.FONT_HERSHEY_PLAIN,
                2, (255, 0, 255), 2
        )
    
    print('[INFO] Inference on Test Image is Ended...')
    return activity 



def is_person_present(frame, thresh=1100):
    kernel = None
    global foog
    
    # Apply background subtraction
    fgmask = foog.apply(frame)

    # Get rid of the shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask
    fgmask = cv2.dilate(fgmask, kernel, iterations=4)

    # Detect contours in the frame
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
     
    # Check if there was a contour and the area is somewhat higher than some threshold so we know it's a person and not noise
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > thresh:
            
        # Get the max contour
        cnt = max(contours, key=cv2.contourArea)

        # Draw a bounding box around the person and label it as person detected
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, 'Person Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
        
        return True, frame
        
        
    # Otherwise report there was no one present
    else:
        return False, frame


def send_message(body, info_dict):

    # Your Account SID from twilio.com/console
    account_sid = info_dict['account_sid']

    # Your Auth Token from twilio.com/console
    auth_token  = info_dict['auth_token']


    client = Client(account_sid, auth_token)

    message = client.messages.create(to=info_dict['your_num'], from_=info_dict['trial_num'], body=body)





def calculate_percentage_change(frame1, frame2):
    # Convert frames to grayscale
    gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate absolute difference between the two frames
    abs_diff = cv2.absdiff(gray_frame1, gray_frame2)

    # Calculate percentage change
    percentage_change = (np.count_nonzero(abs_diff) / abs_diff.size) * 100

    return percentage_change


def save_frame(frame_heap): 
    frame_index = 0
    while frame_heap:
        # Pop the frame with the maximum percentage change (negated)
        neg_change_percentage, frame , date = heapq.heappop(frame_heap)
        change_percentage = -neg_change_percentage
        file_path = 'DetectedPerson'
        # Save the frame to disk
        cv2.imwrite(f'{file_path}/{date}.jpg', frame)
        frame_index += 1
    print("saved frame succesfully")

print('no error in functions')


Libraries succesfully imported
Hello
{0: 'Pushkar', 1: 'Sairaj', 2: 'Tushar'}
no error in functions


In [12]:
# Set Window normal so we can resize it
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# This is a test video
# cap = cv2.VideoCapture("sample_video1.mp4")

# Read the video stream from the camera
cap = cv2.VideoCapture('http://192.168.43.1:8080/video')

# Get width and height of the frame
width = int(cap.get(3))
height = int(cap.get(4))

# Read and store the credentials information in a dict
with open('credential.txt', 'r') as myfile:
  data = myfile.read()

info_dict = eval(data)

# Initialize the background Subtractor
foog = cv2.createBackgroundSubtractorMOG2(detectShadows=True, varThreshold=100, history=2000)

# Status is True when person is present and False when the person is not present.
status = False

# After the person disappears from view, wait at least 7 seconds before making the status False
patience = 0.099

# We don't consider an initial detection unless it's detected 15 times, this gets rid of false positives
detection_thresh = 5

# Initial time for calculating if patience time is up
initial_time = None



# We are creating a deque object of length detection_thresh and will store individual detection statuses here
de = deque([False] * 10, maxlen=10)

# Initialize an empty heap to store frames along with their negated percentage change
frame_heap = []


# Initialize these variables for calculating FPS
fps = 0 
frame_counter = 0
start_time = time.time()
counter = 0

ret,prevframe = cap.read()
initial_frame = prevframe
recognisedName = ""
sent  = False
activity = []

while True:
    
    ret, frame = cap.read()
    if not ret:
        break 
    
    print(recognisedName)
    # This function will return a boolean variable telling if someone was present or not, it will also draw boxes if it 
    # finds someone
    detected, annotated_image = is_person_present(frame)  
    # print(recognisedName)
    # print('Hello')
    

    if detected and len(recognisedName) == 0: 
        # result = recogniseFromFace(frame)  
        print('inside block')
        result = recognize_faces(frame)  
        # print(result)
        activties = recognisize_pose(frame)
        # print(activties)
        activity.append(activties) 
        name , frame = result
        print('inside block')
        if name is not None: 
            if name: 
                recognisedName = name 
            else: 
                recognisedName = ""
    # Register the current detection status on our deque object
    de.appendleft(detected)
    
    # If we have consecutively detected a person 15 times then we are sure that someone is present    
    # We also make this is the first time that this person has been detected so we only initialize the videowriter once
    if sum(de) == detection_thresh and not status:                       
            status = True
            entry_time = datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")
            # out = cv2.VideoWriter('outputs/{}.mp4'.format(entry_time), cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))

    # If status is True but the person is not in the current frame
    if status and not detected:
        
        # Restart the patience timer only if the person has not been detected for a few frames so we are sure it wasn't a 
        # False positive
        print('i reached deepest of the darkest point')
        if sum(de) > (detection_thresh/2): 
            if initial_time is None:
                print('initial time as been setted to',initial_time)
                initial_time = time.time()
                
            
            elif initial_time is not None:        
                print('now the initial time is not null beacuse current time is ',initial_time)
                print(time.time() - initial_time)
                # If the patience has run out and the person is still not detected then set the status to False
                # Also save the video by releasing the video writer and send a text message.
                if  time.time() - initial_time >= patience:
                    status = False
                    print("inside the loop")
                    exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
                    # out.release()
                    # print(out)
                    initial_time = None
                    de = deque([False] * len(de))
                    element_counts = Counter(activity)
                    
                    # Find the most common element
                    most_common_element = element_counts.most_common(1)
                    if most_common_element:
                        most_common_element = most_common_element[0][0]
                        # Do something with most_common_element
                    else:
                        # Handle the case when there are no elements
                        most_common_element = 'unknown activites'
                    body = "Alert: \nA Person Entered the Room at {} \nLeft the room at {} and did ".format(entry_time, exit_time,most_common_element) 
                    # Use Counter to count occurrences of each element
                    activity.clear()
                    if len(recognisedName) > 0: 
                        body = "Alert: {} Has entered in your Room at {} \n Left the room at {} and did {}".format(recognisedName,entry_time,exit_time, most_common_element)
                    print('message has been sent')
                    send_message(body, info_dict)
                    frame_index = 0
                    cv2.destroyWindow("frame")
                    while frame_heap:
                        # Pop the frame with the maximum percentage change (negated)
                        neg_change_percentage, frame , date = heapq.heappop(frame_heap)
                        change_percentage = -neg_change_percentage
                        file_path = 'DetectedPerson'
                        
                        # Save the frame to disk
                        cv2.imwrite(f'{file_path}/{date}.jpg', frame)
                        frame_index += 1
                    print("saved frame succesfully")
                    recognisedName = "" 
                    activity = []
                    continue
                    
                
    
    # If a significant amount of detections (more than half of detection_thresh) has occurred then we reset the Initial Time.
    elif status and sum(de) > (detection_thresh/2):
        change_percentage = calculate_percentage_change(prevframe, frame)
        change_initial_change = calculate_percentage_change(initial_frame,frame)
        # print(change_percentage)
        if change_percentage > 95 and sent == False: 
            sent = True
            body = "Alert: \n Some one has changed the camera configuration at:{}".format(datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")) 
            if recognisedName != 'Unknown': 
                body = "Alert: \n {} has changed the camera configuration at:{}".format(recognisedName, datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")) 
            send_message(body,info_dict)
    
        if change_initial_change > 60: 
            heapq.heappush(frame_heap, (-change_percentage, frame,datetime.datetime.now().strftime("%A, %I-%M-%S %p %d %B %Y")))
        prevframe = frame
        initial_time = None
    
    # Get the current time in the required format
    current_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")

    # Display the FPS
    cv2.putText(annotated_image, 'FPS: {:.2f}'.format(fps), (510, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155), 2)
    
    # Display Time
    cv2.putText(annotated_image, current_time, (310, 20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1)    
    
    # Display the Room Status
    cv2.putText(annotated_image, 'Room Occupied: {}'.format(str(status)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
                (200, 10, 150), 2)

    # Show the patience Value
    if initial_time is None:
        text = 'Patience: {}'.format(patience)
    else: 
        text = 'Patience: {:.2f}'.format(max(0, patience - (time.time() - initial_time)))
        
    cv2.putText(annotated_image, text, (10, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155), 2)   

 
    # Show the Frame
    cv2.imshow('frame', frame)
    
    # Calculate the Average FPS
    frame_counter += 1
    fps = (frame_counter / (time.time() - start_time))
    
    
    # Exit if q is pressed.
    if cv2.waitKey(30) == ord('q'):
        break


# Release Capture and destroy windows
cap.release()
cv2.destroyAllWindows()

while frame_heap:
    # Pop the frame with the maximum percentage change (negated)
    neg_change_percentage, frame , date = heapq.heappop(frame_heap)
    change_percentage = -neg_change_percentage
    file_path = 'DetectedPerson'
                        
    # Save the frame to disk
    cv2.imwrite(f'{file_path}/{date}.jpg', frame)
    frame_index += 1
    print("saved frame succesfully")









inside block
[INFO] Inference on Test Image is Ended...
inside block







inside block
[INFO] Inference on Test Image is Ended...
inside block

inside block
[INFO] Inference on Test Image is Ended...
inside block

inside block
[INFO] Inference on Test Image is Ended...
inside block






















inside block
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
predicted
predictions:  [0.08939882 0.86259836 0.02882284 0.01917993]
predicted Pose Class:  Standing
[INFO] Inference on Test Image is Ended...
inside block






























inside block
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
predicted
predictions:  [0.22426225 0.6911128  0.04584526 0.03877966]
predicted Pose Class:  Standing
[INFO] Inference on Test Image is Ended...
inside block

inside block
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
predicted
predictions:  [0.09881219 0.85404944 0.03410117 0.01303716]
predicted Pose Class:  Standing
[INFO] Inference on Test Image is Ended...
inside block

inside block
1/1 ━━━━━━━━━

In [3]:
mp.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 60, 60, 32)          │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 26, 26, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 10816)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │         692,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,238,539 (8.54 MB)

 Trainable params: 746,179 (2.85 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,492,360 (5.69 MB)